In [5]:
#import necessary tools
import re
from string import digits
import pandas as pd
import numpy as np

In [16]:
#read the documents
with open('data/209_2.txt', 'rb') as f:
    contents = f.read()
    contents = contents.decode("utf-16").rstrip("\n")
    contents = contents.split("\r\n") #maybe need to strip \t?

In [17]:
BlockNum=[]
Procedure=list()
TrialNUM =[1] #eprime writes as TrialNUM
img1 =[]
img2=[]
Choice_RESP = []
Choice_RT=[]


#loop through each line
for line in range(len(contents)-20):
    
    #this line of "running: listlike/black" is in the fixed position of each of 4 conditions (b1b2l1l2)
    #therefore, we use it as our anchor to locate meaningful blocks

    like_check = contents[line+9].startswith("\tRunning: ListLike") == True
    black_check = contents[line+9].startswith("\tRunning: ListBlack") == True
    
    if contents[line] == '\t*** LogFrame Start ***' and (like_check or black_check):
        
        #loop inside the meaningful block                
        for inner_line in range(20):
            
            new_line = line + inner_line
            
            #mark block type
            if contents[new_line].startswith("\tProcedure: "):
                Procedurea = contents[new_line].split("Procedure: ",1)[1] 
                Procedure.append(Procedurea)

            #mark image1
            if contents[new_line].startswith("\timg1: "):
                img1a = contents[new_line].split("img1: ",1)[1] 
                img1.append(img1a)
            
            #mark image2
            if contents[new_line].startswith("\timg2: "):
                img2a = contents[new_line].split("img2: ",1)[1] 
                img2.append(img2a)
                
            #mark block number
            if contents[new_line].startswith("\tBlockNum: "):
                BlockNuma = contents[new_line].split("BlockNum: ",1)[1]
                BlockNuma =[''.join(c for c in BlockNuma if c in digits)]
                BlockNum=BlockNum+BlockNuma
                
            #mark trial number    -> maybe some calculation? such as 50* if ends in 2, or just plus one
            if contents[new_line].startswith("\tTrialNUM: "):
                #or just +=1
                TrialNUMa = TrialNUM[-1] + 1
                
                #TrialNUMa = contents[new_line].split("TrialNUM: ",1)[1]
                #TrialNUMa =[''.join(c for c in TrialNUMa if c in digits)]
                #TrialNUMa = list([float(i) for i in TrialNUMa])
                TrialNUM.append(TrialNUMa)
            
            
            #mark choice Black response 
            if contents[new_line].startswith("\tChoiceB.RESP: ") and contents[new_line].split("\tChoiceB.RESP: ",1) != ['', '']:
                Choice_RESPa = contents[new_line].split("\tChoiceB.RESP: ",1)[1]
                Choice_RESPa =[''.join(c for c in Choice_RESPa if c in digits)]
                Choice_RESP=Choice_RESP+Choice_RESPa
                
            #mark choice Like response 
            if contents[new_line].startswith("\tChoiceL.RESP: ") and contents[new_line].split("\tChoiceL.RESP: ",1) != ['', '']:
                #print(contents[new_line].split("\tChoiceL.RESP: ",1))
                Choice_RESPa = contents[new_line].split("\tChoiceL.RESP: ",1)[1]
                Choice_RESPa =[''.join(c for c in Choice_RESPa if c in digits)]
                Choice_RESP=Choice_RESP+Choice_RESPa
                
            #mark choice Black response time
            if contents[new_line].startswith("\tChoiceB.RT: ") and contents[new_line].split("\tChoiceB.RT: ",1) != ['']:
                Choice_RTa = contents[new_line].split("\tChoiceB.RT: ",1)[1]
                Choice_RTa =[''.join(c for c in Choice_RTa if c in digits)]
                Choice_RTa = list([float(i) for i in Choice_RTa])
                Choice_RT=Choice_RT+Choice_RTa
                
            #mark choice Like response time
            if contents[new_line].startswith("\tChoiceL.RT: ") and contents[new_line].split("\tChoiceL.RT: ",1) != ['']:
                
                Choice_RTa = contents[new_line].split("\tChoiceL.RT: ",1)[1]
                Choice_RTa =[''.join(c for c in Choice_RTa if c in digits)]
                Choice_RTa = list([float(i) for i in Choice_RTa])
                Choice_RT=Choice_RT+Choice_RTa


#Assign variable names
Procedure_df = pd.DataFrame(Procedure)
Procedure_df.columns = ['Btype']  
        
img1 = [int(i) for i in img1] 
img1_df = pd.DataFrame(img1)
img1_df.columns = ['img1']

img2 = [int(i) for i in img2] 
img2_df = pd.DataFrame(img2)
img2_df.columns = ['img2']

TrialNUM = [int(i) for i in TrialNUM] 
TrialNUM_df = pd.DataFrame(TrialNUM)
TrialNUM_df.columns = ['trial']

BlockNum = [int(i) for i in BlockNum] 
BlockNum_df = pd.DataFrame(BlockNum)
BlockNum_df.columns = ['block']

Choice_RESP = [int(i) for i in Choice_RESP] 
Choice_RESP_df = pd.DataFrame(Choice_RESP)
Choice_RESP_df.columns = ['RESP']

Choice_RT = [int(i) for i in Choice_RT] 
Choice_RT_df = pd.DataFrame(Choice_RT)
Choice_RT_df.columns = ['RT']


In [18]:
#merge the data
sub_id = contents[16].split("Subject: ",1)[1] 
sub_df = pd.DataFrame(np.repeat(sub_id, img1_df.shape[0], axis = 0))
sub_df.columns = ['sub']

all = pd.concat([sub_df.reset_index(drop=True), BlockNum_df], axis=1)
all = pd.concat([all.reset_index(drop=True), Procedure_df], axis=1)
all = pd.concat([all.reset_index(drop=True), TrialNUM_df], axis=1)
all = pd.concat([all.reset_index(drop=True), img1_df], axis=1)
all = pd.concat([all.reset_index(drop=True), img2_df], axis=1)
all = pd.concat([all.reset_index(drop=True), Choice_RESP_df], axis=1)
all = pd.concat([all.reset_index(drop=True), Choice_RT_df], axis=1)

all

,sub,block,Btype,trial,img1,img2,RESP,RT
0,209,2.0,ChooseLike,1,163.0,236.0,2.0,2642.0
1,209,2.0,ChooseLike,2,190.0,201.0,1.0,2376.0
2,209,2.0,ChooseLike,3,135.0,205.0,2.0,1248.0
3,209,2.0,ChooseLike,4,139.0,212.0,1.0,1947.0
4,209,2.0,ChooseLike,5,146.0,149.0,1.0,1873.0
...,...,...,...,...,...,...,...,...
96,209,3.0,ChooseBlack,97,222.0,223.0,2.0,867.0
97,209,3.0,ChooseBlack,98,161.0,173.0,1.0,925.0
98,209,3.0,ChooseBlack,99,142.0,180.0,2.0,1934.0
99,209,3.0,ChooseBlack,100,135.0,187.0,2.0,1473.0


In [19]:
#remove last NA line if needed

all_all= all[:-1]
all_all

,sub,block,Btype,trial,img1,img2,RESP,RT
0,209,2.0,ChooseLike,1,163.0,236.0,2.0,2642.0
1,209,2.0,ChooseLike,2,190.0,201.0,1.0,2376.0
2,209,2.0,ChooseLike,3,135.0,205.0,2.0,1248.0
3,209,2.0,ChooseLike,4,139.0,212.0,1.0,1947.0
4,209,2.0,ChooseLike,5,146.0,149.0,1.0,1873.0
...,...,...,...,...,...,...,...,...
95,209,3.0,ChooseBlack,96,124.0,223.0,1.0,616.0
96,209,3.0,ChooseBlack,97,222.0,223.0,2.0,867.0
97,209,3.0,ChooseBlack,98,161.0,173.0,1.0,925.0
98,209,3.0,ChooseBlack,99,142.0,180.0,2.0,1934.0


In [20]:

#export
all_all.to_excel("choice_f_209_2.xlsx")

#has wrong first column, needs to manually remove